In [19]:
!pip install kaggle
!mkdir -p ~/.kaggle

In [ ]:
%%writefile ~/.kaggle/kaggle.json
{
  "username": "rashvin20",
  "key": "replace your key here"
}


Writing /root/.kaggle/kaggle.json


In [21]:
!chmod 600 ~/.kaggle/kaggle.json

In [22]:
!kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
 98% 4.67G/4.78G [00:10<00:00, 340MB/s]
100% 4.78G/4.78G [00:10<00:00, 486MB/s]


In [23]:
!unzip fruits.zip -d fruits_dataset/

Streaming output truncated to the last 5000 lines.
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_21.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_213.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_217.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_221.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_225.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_229.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_233.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_237.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-36

In [24]:
!pip install -q wandb torchviz fvcore grad-cam scikit-learn graphviz
!apt-get -q install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Reading package lists...
Building dependency tree...
Reading state information...
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [25]:
import os, random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image

from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from fvcore.nn import FlopCountAnalysis
from torchviz import make_dot

import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

os.makedirs("/content/outputs", exist_ok=True)


Running on: cuda


In [26]:
wandb.login()

wandb.init(
    project="vision-cw",
    name="cnn_l1_l1",
    config={
        "dataset": "Fruits-360",
        "augmentation": True,
        "epochs": 30,
        "batch_size": 32,
        "lr": 1e-3
    }
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rashvin-rajanah-23 (rashvin-rajanah-23-ucl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import os, random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from PIL import Image

import wandb
from fvcore.nn import FlopCountAnalysis
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# ======================================================
# CONFIG
# ======================================================
device = "cuda" if torch.cuda.is_available() else "cpu"

EPOCHS = 20
PATIENCE = 5
LR = 1e-3

LAMBDA_L1 = 1e-5
LAMBDA_L2 = 1e-4

OUTPUT_DIR = "/content/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

#load path here
train_path = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Training"
val_path   = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation"

transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])

train_dataset_full = datasets.ImageFolder(train_path, transform=transform)
val_dataset_full   = datasets.ImageFolder(val_path, transform=transform)

num_classes = len(train_dataset_full.classes)
print("Classes:", num_classes)

NUM_TRAIN = 10000
NUM_VAL   = 2000

train_idx = random.sample(range(len(train_dataset_full)), NUM_TRAIN)
val_idx   = random.sample(range(len(val_dataset_full)), NUM_VAL)

train_dataset = Subset(train_dataset_full, train_idx)
val_dataset   = Subset(val_dataset_full, val_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

#custom model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 12 * 12, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(self.features(x))

#training func
def train_model(reg_type):
    model = CNN(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()

    if reg_type == "l2":
        optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=LAMBDA_L2)
    else:
        optimizer = optim.Adam(model.parameters(), lr=LR)

    best_val_loss = float("inf")
    wait = 0

    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(EPOCHS):
        print(f"\n[{reg_type.upper()}] Epoch {epoch+1}/{EPOCHS}")
        model.train()
        correct, total, running_loss = 0, 0, 0

        for x, y in tqdm(train_loader, leave=False):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            out = model(x)
            loss = criterion(out, y)

            if reg_type == "l1":
                l1_penalty = sum(p.abs().sum() for p in model.parameters())
                loss = loss + LAMBDA_L1 * l1_penalty

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)

        train_losses.append(running_loss / len(train_loader))
        train_accs.append(correct / total)

        model.eval()
        v_loss, v_correct, v_total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                v_loss += criterion(out, y).item()
                v_correct += (out.argmax(1) == y).sum().item()
                v_total += y.size(0)

        val_losses.append(v_loss / len(val_loader))
        val_accs.append(v_correct / v_total)
        #early stop
        if val_losses[-1] < best_val_loss:
            best_val_loss = val_losses[-1]
            torch.save(model.state_dict(), f"{OUTPUT_DIR}/best_{reg_type}.pth")
            wait = 0
        else:
            wait += 1
            if wait >= PATIENCE:
                break

    return model, train_losses, val_losses, train_accs, val_accs


model_l2, tr_l2, va_l2, ta_l2, vaa_l2 = train_model("l2")
model_l1, tr_l1, va_l1, ta_l1, vaa_l1 = train_model("l1")

#plot code here
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(tr_l2, label="Train L2")
plt.plot(va_l2, label="Val L2")
plt.plot(tr_l1, "--", label="Train L1")
plt.plot(va_l1, "--", label="Val L1")
plt.title("Loss Comparison")
plt.legend()

plt.subplot(1,2,2)
plt.plot(ta_l2, label="Train L2")
plt.plot(vaa_l2, label="Val L2")
plt.plot(ta_l1, "--", label="Train L1")
plt.plot(vaa_l1, "--", label="Val L1")
plt.title("Accuracy Comparison")
plt.legend()

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/l1_vs_l2_comparison.jpg", dpi=300)
plt.close()

print("✅ L1 vs L2 comparison complete")


Classes: 116

[L2] Epoch 1/20



[L2] Epoch 2/20



[L2] Epoch 3/20



[L2] Epoch 4/20



[L2] Epoch 5/20


...


[L1] Epoch 17/20



[L1] Epoch 18/20



[L1] Epoch 19/20



[L1] Epoch 20/20


✅ L1 vs L2 comparison complete
